<a href="https://colab.research.google.com/github/jeaneigsi/Humain-/blob/main/OCR_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning / test pipeline

In [ ]:
!pip install memory_profiler
!pip -q install datasets
!pip -q install paddlepaddle
!pip -q install "paddleocr>=2.0.1" # Recommend to use version 2.0.1+

In [ ]:
!export GROQ_API_KEY=


In [ ]:
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.0 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq

def initialize_groq_client():
    """
    Initialize the Groq client using the API key from environment variables.
    """
    api_key ="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"
    if not api_key:
        raise ValueError("GROQ_API_KEY environment variable not set.")
    return Groq(api_key=api_key)

def get_chat_completion(query, model="llama3-70b-8192"):
    """
    Get a chat completion from the Groq API based on the user query.

    :param query: User's query as a string.
    :param model: Model to be used for the completion.
    :return: The response content from the chat completion.
    """
    client = initialize_groq_client()
    messages = [
        {
            "role": "system",
            "content": "You are an assistant that takes a piece of text that has been corrupted during OCR digitisation, and produce a corrected version of the same text. Dont add any comment, explanation or other. piece can contains informations from tables, figures ... MAKE IT READABLE, UNDERSTABLE AND CLEAR. WRITE IN MARKDOWN"
        },
        {
            "role": "user",
            "content": query,
        }
    ]
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model
    )
    return chat_completion.choices[0].message.content

In [ ]:
user_query="""5.Cash Flow Statement The cash flow from operating activities has improved by $12,234,560, driven primarily by increa The cash low rm operating aciis asimpod by se2s4,5o den primal y increase activities shows a substantial outflow of $50,345,890 due to major capital expenditures in upgrading the company's IT infrastructure. 6.Key Performance Indicators (KPls) Return on Equity (ROE): 22.38% Return on Assets (ROA): 13.58% Current Ratio:1.87 Quick Ratio:1.34 Debt-to-Equity Ratio:0.65 I.Tutuleoutiook The company anticipates continued growth in the upcoming fiscal year, targeting a 15% increase in. gross revenue. Strategic investments in Al-driven technologies are expected to yield a 20% improvement in operational efficiency, resulting in higher profit margins.."""

In [ ]:
response_content = get_chat_completion(user_query)


In [ ]:
response_content

"### 5. Cash Flow Statement\nThe cash flow from operating activities has improved by $12,234,560, driven primarily by increase. The cash flow from operating activities shows a substantial outflow of $50,345,890 due to major capital expenditures in upgrading the company's IT infrastructure.\n\n### 6. Key Performance Indicators (KPIs)\n#### Return on Equity (ROE): 22.38%\n#### Return on Assets (ROA): 13.58%\n#### Current Ratio: 1.87\n#### Quick Ratio: 1.34\n#### Debt-to-Equity Ratio: 0.65\n\n### IT Outlook\nThe company anticipates continued growth in the upcoming fiscal year, targeting a 15% increase in gross revenue. Strategic investments in AI-driven technologies are expected to yield a 20% improvement in operational efficiency, resulting in higher profit margins."

In [ ]:
from datasets import load_dataset

data=load_dataset("vidore/shiftproject_test_ocr_chunk",split='test')
data

In [ ]:
data=data.select_columns(['image','text_description'])
data[0]

Dataset({
    features: ['image', 'text_description'],
    num_rows: 2051
})

In [ ]:
data[0]['image']

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1654x2339>,
 'text_description': "La taille moyenne des champs pétroliers découverts au Kazakhstan est en déclin depuis les années 1970.\n\nLa taille moyenne a augmenté légérement au cours des années 2000 de concert avec le nouveau cycle de découvertes lié aux champs offshore. Cependant, ces chiffres ont repris rapidement leur déclin rejoignant la tendance générale. Depuis 2007, seules deux années présentent une valeur supérieure a 25 millions de barils.\n\nIl. Historique de production\n\nLa production d’hydrocarbures liquides du Kazakhstan est en forte hausse depuis le début des années 1990. En 2019 elle représente 710 millions de barils (1,1 Mb/j). Cette production se trouvait depuis 1977 sur un plateau d’environ 150 millions de barils par an. La production d’hydrocarbures du Kazakhstan reste principalement située a terre (84 % en 2019). Une partie de cette production est issue depuis 2016 de champs se situant en eaux p

In [ ]:
!wget https://ds4sd-icdar23-doclaynet-competition.s3.eu-de.cloud-object-storage.appdomain.cloud/competition_dataset_groundtruth.zip
!unzip /content/JPEG.zip

In [ ]:
from paddleocr import PaddleOCR # Import the PaddleOCR class

ocr = PaddleOCR(use_angle_cls=True, lang='en')

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:17<00:00, 235kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:11<00:00, 915kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:14<00:00, 147kiB/s]

[2024/08/06 15:43:54] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [ ]:
import numpy as np
def pil_to_np(pil_image):
    # Convert PIL Image to NumPy Array
    return np.array(pil_image)

In [ ]:
ds['validation']

In [ ]:
test_datasets=ds['test']
test_datasets

In [ ]:
valid_datasets=ds['validation']
valid_datasets

In [ ]:
slice_params = {
    'horizontal_stride': 300,  # Horizontal stride for sliding window
    'vertical_stride': 500,    # Vertical stride for sliding window
    'merge_x_thres': 50,       # Threshold for merging slices horizontally
    'merge_y_thres': 35        # Threshold for merging slices vertically
}

In [ ]:
from paddleocr import PaddleOCR
from memory_profiler import profile

@profile
def ocr_pipeline(img_path):
  if isinstance(img_path,str):
    img_path=img_path
  else:
      img_path=pil_to_np(img_path)
  result=ocr.ocr(img_path, cls=True,det=True,slice=slice_params)
  for idx in range(len(result)):
    res = result[idx]
    txts = [line[1][0] for line in res]
    final=" ".join(txts)
    print(" ".join(txts))
    return final


In [ ]:
ocr_text=ocr_pipeline("/content/pose3.jpeg")
ocr_text

ERROR: Could not find file <ipython-input-8-4fa9bb6a2b82>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
[2024/08/06 17:50:33] ppocr DEBUG: dt_boxes num : 15, elapsed : 0.2941744327545166
[2024/08/06 17:50:33] ppocr DEBUG: cls num  : 15, elapsed : 0.06325984001159668
[2024/08/06 17:50:35] ppocr DEBUG: rec_res num  : 15, elapsed : 2.2539145946502686
5.Cash Flow Statement The cash flow from operating activities has improved by $12,234,560, driven primarily by increa The cash low rm operating aciis asimpod by se2s4,5o den primal y increase activities shows a substantial outflow of $50,345,890 due to major capital expenditures in upgrading the company's IT infrastructure. 6.Key Performance Indicators (KPls) Return on Equity (ROE): 22.38% Return on Assets (ROA): 13.58% Current Ratio:1.87 Quick Ratio:1.34 Debt-to-Equity Ratio:0.65 I.Tutuleoutiook The company anticipates continued growth in the upcoming fiscal year, targeting a 15% in

"5.Cash Flow Statement The cash flow from operating activities has improved by $12,234,560, driven primarily by increa The cash low rm operating aciis asimpod by se2s4,5o den primal y increase activities shows a substantial outflow of $50,345,890 due to major capital expenditures in upgrading the company's IT infrastructure. 6.Key Performance Indicators (KPls) Return on Equity (ROE): 22.38% Return on Assets (ROA): 13.58% Current Ratio:1.87 Quick Ratio:1.34 Debt-to-Equity Ratio:0.65 I.Tutuleoutiook The company anticipates continued growth in the upcoming fiscal year, targeting a 15% increase in. gross revenue. Strategic investments in Al-driven technologies are expected to yield a 20% improvement in operational efficiency, resulting in higher profit margins.."

In [ ]:
ocr_text

"5.Cash Flow Statement The cash flow from operating activities has improved by $12,234,560, driven primarily by increa The cash low rm operating aciis asimpod by se2s4,5o den primal y increase activities shows a substantial outflow of $50,345,890 due to major capital expenditures in upgrading the company's IT infrastructure. 6.Key Performance Indicators (KPls) Return on Equity (ROE): 22.38% Return on Assets (ROA): 13.58% Current Ratio:1.87 Quick Ratio:1.34 Debt-to-Equity Ratio:0.65 I.Tutuleoutiook The company anticipates continued growth in the upcoming fiscal year, targeting a 15% increase in. gross revenue. Strategic investments in Al-driven technologies are expected to yield a 20% improvement in operational efficiency, resulting in higher profit margins.."

In [ ]:
!pip install editdistance
!pip install jiwer

In [ ]:
import editdistance
from jiwer import wer

def calculate_metrics(true_text, ocr_text, ocr_llm_text):
    # Calculate Character Error Rate (CER)
    cer_ocr = editdistance.eval(ocr_text, true_text) / len(true_text)
    cer_ocr_llm = editdistance.eval(ocr_llm_text, true_text) / len(true_text)

    # Calculate Word Error Rate (WER)
    wer_ocr = wer(true_text, ocr_text)
    wer_ocr_llm = wer(true_text, ocr_llm_text)

    # Calculate Levenshtein Distance
    lev_ocr = editdistance.eval(ocr_text, true_text)
    lev_ocr_llm = editdistance.eval(ocr_llm_text, true_text)

    return {
        "CER_OCR": cer_ocr,
        "CER_OCR_LLM": cer_ocr_llm,
        "WER_OCR": wer_ocr,
        "WER_OCR_LLM": wer_ocr_llm,
        "Levenshtein_OCR": lev_ocr,
        "Levenshtein_OCR_LLM": lev_ocr_llm
    }


In [ ]:

# Example usage:
true_text = "This is the correct text."
ocr_llm_text = "This is the correct txt."

metrics = calculate_metrics(true_text, ocr_text, ocr_llm_text)
print(metrics)


{'CER_OCR': 0.08, 'CER_OCR_LLM': 0.04, 'WER_OCR': 0.4, 'WER_OCR_LLM': 0.2, 'Levenshtein_OCR': 2, 'Levenshtein_OCR_LLM': 1}


# Flask deploiement

In [ ]:
from flask import Flask, request, jsonify
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np
import io

app = Flask(__name__)

In [ ]:
slice_params = {
    'horizontal_stride': 300,  # Horizontal stride for sliding window
    'vertical_stride': 500,    # Vertical stride for sliding window
    'merge_x_thres': 50,       # Threshold for merging slices horizontally
    'merge_y_thres': 35        # Threshold for merging slices vertically
}
# cut an image in multiple pieces , add ocr to

In [ ]:


ocr = PaddleOCR(use_angle_cls=True, lang='en')
slice_params = {'horizontal_stride': 300, 'vertical_stride': 500, 'merge_x_thres': 50, 'merge_y_thres': 35}

def pil_to_np(img):
    return np.array(img)

@app.route('/ocr', methods=['POST'])
def ocr_pipeline():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image file provided'}), 400

        img_file = request.files['image']
        img = Image.open(img_file.stream)
        img_np = pil_to_np(img)
        result = ocr.ocr(img_np, cls=True, det=True, slice=slice_params)
        all_texts = []
        for res in result:
            txts = [line[1][0] for line in res]
            all_texts.append(" ".join(txts))

        final_text = " ".join(all_texts)

        return jsonify({'text': final_text}), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500




In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)